In [1]:
import xarray as xr
import datetime as dt
import pandas as pd
import numpy as np
adir_adcp = 'F:/data/cruise_data/saildrone/baja-2018/adcp_data/'
adir = 'F:/data/cruise_data/saildrone/baja-2018/'

filename_loggers_usv=adir + 'saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1_withloggers.nc'
filename_adcp = adir_adcp + 'saildrone-gen_4-baja_2018-sd1002-adcp-20180411T175730-20180611T235730_300s-v1.nc'

ds_usv = xr.open_dataset(filename_loggers_usv)  #1 minute averaging
ds_usv.close()
ds_adcp = xr.open_dataset(filename_adcp)            #10 minute sampling
ds_adcp.close()
offset=0

ds_adcp=ds_adcp.rename({'latitude':'lat','longitude':'lon'})
ds_adcp=ds_adcp.swap_dims({'obs':'time'})

ds_usv=ds_usv.rename({'latitude':'lat','longitude':'lon'})
ds_usv=ds_usv.swap_dims({'obs':'time'})
lon = ds_usv.lon
lat = ds_usv.lat
del ds_usv['lon']
del ds_usv['lat']
ds_usv['lon'] = ('time', lon)
ds_usv['lat'] = ('time', lat)   


In [2]:
##print(ds_adcp.time[0:10].data)
#print(ds_usv.time[0,0:10].data)
#print(type(ds_adcp.time))
#ds_adcp

[['2018-04-11T19:50:00.000000000']
 ['2018-04-11T20:00:00.000000000']
 ['2018-04-11T20:10:00.000000000']
 ['2018-04-11T20:20:00.000000000']
 ['2018-04-11T20:30:00.000000000']
 ['2018-04-11T20:40:00.000000000']
 ['2018-04-11T20:50:00.000000000']
 ['2018-04-11T21:00:00.000000000']
 ['2018-04-11T21:10:00.000000000']
 ['2018-04-11T21:20:00.000000000']]
['2018-04-11T18:00:00.000000000' '2018-04-11T18:01:00.000000000'
 '2018-04-11T18:02:00.000000000' '2018-04-11T18:03:00.000000000'
 '2018-04-11T18:04:00.000000000' '2018-04-11T18:05:00.000000000'
 '2018-04-11T18:06:00.000000000' '2018-04-11T18:07:00.000000000'
 '2018-04-11T18:08:00.000000000' '2018-04-11T18:09:00.000000000']
<class 'xarray.core.dataarray.DataArray'>


<xarray.Dataset>
Dimensions:              (beam: 4, cell: 50, obs: 8760, trajectory: 1)
Coordinates:
  * trajectory           (trajectory) float32 9.96921e+36
Dimensions without coordinates: beam, cell, obs
Data variables:
    cell_depth           (cell) float32 ...
    latitude             (obs, trajectory) float64 ...
    longitude            (obs, trajectory) float64 ...
    time                 (obs, trajectory) datetime64[ns] ...
    roll                 (obs, trajectory) float64 ...
    pitch                (obs, trajectory) float64 ...
    heading              (obs, trajectory) float64 ...
    nav_end_latitude     (obs, trajectory) float64 ...
    nav_end_longitude    (obs, trajectory) float64 ...
    nav_end_time         (obs, trajectory) datetime64[ns] ...
    nav_start_latitude   (obs, trajectory) float64 ...
    nav_start_longitude  (obs, trajectory) float64 ...
    nav_start_time       (obs, trajectory) datetime64[ns] ...
    bt_vel_east          (obs, trajectory) float64 .

In [3]:
#ds_usv2=ds_usv.sel(trajectory=1002)

#ds_usv2.time
#ds_usv2.coords('time',ds_usv2('time'))
#ds_temds.coords['day'] = ('time', [6, 7, 8])
#ds_usv2.assign_coords(obs=ds_usv2.time)
#ds_usv2


#this calculates hourly data that only uses the last 8 minutes of each hour for the average
sub = ds_usv.copy(deep=True)
pt = pd.to_datetime(sub.time.data)
pmin=pt.minute
psec=pt.second
tem=sub.where(pmin>=52)  #data average should be only use min 52  to :60 
ds_usv_10min = tem.resample(time='1h', skipna=True, label='right').mean()



In [4]:
tdim=ds_adcp.time[:,0].shape
tdim
print(ds_adcp.time)

<xarray.DataArray 'time' (obs: 8760, trajectory: 1)>
array([['2018-04-11T19:50:00.000000000'],
       ['2018-04-11T20:00:00.000000000'],
       ['2018-04-11T20:10:00.000000000'],
       ...,
       ['2018-06-11T19:50:00.000000000'],
       ['2018-06-11T20:00:00.000000000'],
       ['2018-06-11T20:10:00.000000000']], dtype='datetime64[ns]')
Coordinates:
  * trajectory  (trajectory) float32 9.96921e+36
Dimensions without coordinates: obs
Attributes:
    long_name:           Time
    standard_name:       time
    vendor_name:         VectorNav
    installed_date:      2017-06-01T00:00:00Z
    model_product_page:  http://www.vectornav.com/products/vn200-smd
    device_name:         VectorNav IMU (N/A)


In [ ]:
istart=0
print(tdim[0])
for i in range(0,tdim[0]):
    center_time=ds_adcp.time[i,0]
    upper_time=center_time+np.timedelta64(150,'s')
    lower_time=center_time-np.timedelta64(150,'s')
    #print(lower_time.data,upper_time.data)
    ds_usv2=ds_usv.where((ds_usv.time>lower_time) & (ds_usv.time<=upper_time)).mean() #,upper_time))
    if istart == 0:
        ds_usv3=ds_usv2
        istart=1
    else:
        ds_usv3=xr.concat([ds_usv3, ds_usv2], 'obs')

8760


In [ ]:
ds_usv3

In [6]:
filename_10min=adir + 'saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-10_minutes-v1_withloggers_matchedadcp.nc'
#ds_usv3.to_netcdf(filename_10min)

In [ ]:
#pt = pd.to_datetime(ds_usv2.time.data)+offset  #add offset seconds
#psec=pt.second
#tem=ds_usv2 #[(psec <=12)]  #data average should be only use sec 54 to sec 06 of each minute 
#tem['time']=tem['time']+offset #resample goes from :00 to :59 so shift +6sec
#print(tem.time)
#xf_avg1 = tem.resample(time='10min').mean()
